[![Renode](https://dl.antmicro.com/projects/renode/renode.svg)](https://renode.io)

[![Run in Google Colab](https://img.shields.io/badge/-Run%20in%20Google%20colab-%23007ded?logo=google-colab&logoColor=white&style=for-the-badge)](https://colab.research.google.com/github/antmicro/test-colabs/blob/main/boards/_hello_world.ipynb) [![View ipynb](https://img.shields.io/badge/-View%20ipynb%20source-%23007ded?logo=jupyter&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/_hello_world.ipynb) [![View Python source](https://img.shields.io/badge/-View%20Python%20source-%23007ded?logo=python&logoColor=white&style=for-the-badge)](https://github.com/antmicro/test-colabs/blob/main/boards/_hello_world.py)

## Install requirements

In [ ]:
! pip install -q git+https://github.com/antmicro/renode-colab-tools.git
! pip install -q git+https://github.com/antmicro/renode-run.git@new-features
! pip install -q git+https://github.com/antmicro/pyrenode.git@renode-run-experiments
! pip install -q robotframework==4.0.1
! renode-run download

## Start Renode

In [ ]:
from pyrenode import connect_renode, get_keywords
connect_renode()
get_keywords()

## Setup a script

In [ ]:
%%writefile script.resc

using sysbus
$name?="black_f407zg_pro"
mach create $name

machine LoadPlatformDescription @https://zephyr-dashboard.renode.io/black_f407zg_pro-hello_world.repl
machine EnableProfiler $ORIGIN/metrics.dump

showAnalyzer sysbus.usart2

macro reset
"""
    sysbus LoadELF @https://zephyr-dashboard.renode.io/black_f407zg_pro-zephyr-hello_world.elf
    cpu0 VectorTableOffset `sysbus GetSymbolAddress "_vector_table"`
"""

runMacro $reset

## Run the sample

In [ ]:
ExecuteCommand("include @script.resc")
CreateTerminalTester("sysbus.usart2", timeout=5)
StartEmulation()

WaitForLineOnUart("Hello World! black_f407zg_pro")
print(ExecuteCommand("sysbus.usart2 DumpHistoryBuffer"))

ResetEmulation()

## Renode metrics analysis

In [ ]:
import sys
from pathlib import Path
sys.path.append(Path('/root/.config/renode/renode-run.path').read_text())

from renode_colab_tools import metrics
from tools.metrics_analyzer.metrics_parser import MetricsParser
metrics.init_notebook_mode(connected=False)
parser = MetricsParser('metrics.dump')

metrics.display_metrics(parser)